<a href="https://colab.research.google.com/github/AmbrogioMB/AlgOpt/blob/main/Introduction_to_Integer_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. A first problem
Suppose we wish to invest $\$19,000$. We have identified four investment opportunities. Investment 1 requires an investment of $\$6,700$ and has a net present value of $\$8,000$; investment 2 requires $\$10,000$ and has a value of $\$11,000$; investment 3 requires $\$5,500$ and has a value of $\$6,000$; and investment 4 requires $\$3,400$ and has a value of $\$4,000$. Into which investments should we place our money so as to maximize our total present value? Each project is a ''take it or leave it'' opportunity: It is not allowed to invest partially in any of the projects. Such problems are called capital budgeting problems. As in linear programming, our first step is to decide on the variables. In this case, it is easy: We will use a $0-1$ variable $x_j$ for each investment. If $x_j$ is $1$ then we will make investment $j$. If it is $0$, we will not make the investment. This leads to the $0-1$ programming problem

In [ ]:
! pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

investments = ['Investment 1', 'Investment 2', 'Investment 3', 'Investment 4']
costs = [6.7, 10, 5.5, 3.4]
values = [8, 11, 6, 4]


model_continuous = gp.Model("example LP")

x = model_continuous.addVars(investments, name="x", vtype=GRB.CONTINUOUS, ub=1)

model_continuous.addLConstr(gp.quicksum(costs[i]*x[investments[i]] for i in range(len(investments))) <= 19, "Constraint1")

model_continuous.setObjective(gp.quicksum(values[i]*x[investments[i]] for i in range(len(investments))), GRB.MAXIMIZE)

model_continuous.optimize()

x_sol = {i: x[i].x for i in investments}
print("Decision variables:")
print(x_sol)

Unfortunately, this solution is not integral. Rounding $x_2$ down to $0$ gives a
feasible solution with a value of $\$12,000$.

In [ ]:
x_round = {i: int(x[i].x) for i in investments}
print("Decision variables (rounded):")
print(x_round)
cost_round = sum(costs[i]*x_round[investments[i]] for i in range(len(investments)))
value_round = sum(values[i]*x_round[investments[i]] for i in range(len(investments)))
print("Objective value rounded solution:", value_round)
print("Cost rounded solution:", cost_round)

There is a better integer solution, however.

In [ ]:
model_integer = gp.Model("example ILP")

x = model_integer.addVars(investments, name="x", vtype=GRB.BINARY)

model_integer.addLConstr(gp.quicksum(costs[i]*x[investments[i]] for i in range(len(investments))) <= 19, "Constraint1")

model_integer.setObjective(gp.quicksum(values[i]*x[investments[i]] for i in range(len(investments))), GRB.MAXIMIZE)

model_integer.optimize()
x_sol = {i: x[i].x for i in investments}
print("Decision variables:")
print(x_sol)
print("Objective value:", model_integer.objVal)

This example shows that rounding does not necessarily give an optimal solution.

There are a number of additional constraints we might want to add. For instance, consider the following constraints:
1. We can only make two investments.
2. If investment 2 is made, then investment 4 must also be made.
3. If investment 1 is made, then investment 3 cannot be made.

All of these, and many more *logical restrictions*, can be enforced using
$0-1$ variables. In these cases, the constraints are the following

In [ ]:
model_integer.addLConstr(gp.quicksum(x[investments[i]] for i in range(len(investments))) <= 2, "Constraint2")
model_integer.addLConstr(x['Investment 2'] - x['Investment 4'] <= 0, "Constraint3")
model_integer.addLConstr(x['Investment 1'] - x['Investment 3'] <= 1, "Constraint4")
model_integer.update()
model_integer.optimize()
x_sol = {i: x[i].x for i in investments}
print("Decision variables:")
print(x_sol)
print("Objective value:", model_integer.objVal)

## 2. Investment in Bond Bundles

Suppose an investor has $\$50,000$ to invest in three types of corporate bonds. Each bond is sold only in bundles of $10$ units, and each bundle has a fixed cost and expected return. The investor can invest in multiple bundles of each type, but only whole bundles can be purchased (no partial bundles). The goal is to maximize total expected return, subject to the budget constraint.

$$
\begin{array}{|c|r|r|r|}
\hline
\textbf{Bond} & \textbf{Cost per bundle} & \textbf{Expected return} & \textbf{Max bundles}  \\
\hline
\text{A} & 6.0 & 0.72 & 5\\
\text{B} & 8.5 & 1.00 & 4\\
\text{C} & 4.0 & 0.44 & 6\\
\hline
\end{array}
$$

In [ ]:
m_bund = gp.Model("Bond bundles")
bonds = ['A', 'B', 'C']
costs = [6, 8.5, 4]
returns = [0.72, 1, 0.44]
max_bundles = [5, 4, 6]
x = m_bund.addVars(bonds, name="x", vtype=GRB.INTEGER, ub=max_bundles)
m_bund.addLConstr(gp.quicksum(costs[i]*x[bonds[i]] for i in range(len(bonds))) <= 50, "Constraint1")
m_bund.setObjective(gp.quicksum(returns[i]*x[bonds[i]] for i in range(len(bonds))), GRB.MAXIMIZE)
m_bund.optimize()
x_sol = {i: x[i].x for i in bonds}
print("Decision variables:")
print(x_sol)
print("Objective value:", m_bund.objVal)

## 3. Exercise: oil exploration
As the leader of an oil exploration drilling venture, you must determine the best selection of 5 out of 10 possible sites. Label the sites $s_1, s_2, \dots, s_{10}$ and the expected profits associated with each as $p_1, p_2, \dots, p_{10}$.

1. If site $s_2$ is explored, then site $s_3$ must also be explored. Furthermore, regional development restrictions are such that
2. Exploring sites $s_1$ and $s_7$ will prevent you from exploring site $s_8$.
3. Exploring sites $s_3$ or $s_4$ will prevent you from exploring site $s_5$.

In [ ]:
import numpy as np
m_oil = gp.Model("Oil drilling")
p = np.random.randint(1, 20, 10)
print('Profits: ', p)
# variables and constraints here

# Decision variables: x[i] = 1 if site si is selected, else 0
x = m_oil.addVars(10, vtype=GRB.BINARY, name="x")

# Constraint: Select exactly 5 sites
m_oil.addConstr(gp.quicksum(x[i] for i in range(10)) == 5, "select_5_sites")

# Constraint: If site 2 is selected, site 3 must be selected → x[1] <= x[2]
m_oil.addConstr(x[1] <= x[2], "s2_implies_s3")

# Constraint: If both site 1 and site 7 are selected, site 8 must not be → x[0] + x[6] + x[7] <= 2
m_oil.addConstr(x[0] + x[6] + x[7] <= 2, "s1_and_s7_block_s8")

# Constraint: If site 3 or site 4 is selected, site 5 must not be selected
# x[4] <= 1 - 0.5*(x[2] + x[3])
m_oil.addConstr(x[4] <= 1 - 0.5*(x[2] + x[3]), "s3_or_s4_block_s5")
# or even better with two constraints x[4] <= 1 - x[2] and x[4] <= 1 - x[3]

# Objective: Maximize total expected profit
m_oil.setObjective(gp.quicksum(p[i] * x[i] for i in range(10)), GRB.MAXIMIZE)

# Optimize model
m_oil.optimize()

# Output selected sites and profits
selected_sites = [f"s{i+1}" for i in range(10) if x[i].X > 0.5]
selected_profits = [int(p[i]) for i in range(10) if x[i].X > 0.5]
print("Selected Sites:", selected_sites)
print("Profits:", selected_profits)
print("Total Profit:", sum(selected_profits))

## 4. Exercise: Sudoku

The game is played on a $9 \times 9$ grid which is subdivided into $9$ blocks of $3 \times 3$ contiguous cells. The grid must be filled with numbers $1, \dots ,9$ so that all the numbers between $1$ and $9$ appear in each row, in each column and in each of the nine blocks. A game consists of an initial assignment of numbers in some cells.

\begin{array}{|ccc|ccc|ccc|}
\hline
  8  &  - &  - &  - & 2  & 6  &  - & -  &  - \\
  -  &  - &  - &  - & -  & -  &  7 & -  &  4 \\
  -  &  - &  - &  7 & -  & -  &  - & -  &  5 \\
  \hline
  -  &  - &  - &  1 & -  & -  &  - & 3  &  6 \\
  -  &  1 &  - &  - & 8  & -  &  - & 4  &  - \\
  9  &  8 &  - &  - & -  & 3  &  - & -  &  - \\
  \hline
  3  &  - &  - &  - & -  & 1  &  - & -  &  - \\
  7  &  - &  5 &  - & -  & -  &  - & -  &  - \\
  -  &  - &  - &  2 & 5  & -  &  - & -  &  8 \\
\hline
\end{array}

This is a decision problem that can be modeled with binary variables
$x_{ijk}$, $1 \leq  i, j, k \leq 9$ where $x_{ijk} = 1$ if number $k$ is entered in position with coordinates $i, j$ of the grid, and $0$ otherwise.

In [ ]:
m_sudoku = gp.Model("Sudoku")
# variables and constraints here
puzzle = [
    [8, 0, 0, 0, 2, 6, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 7, 0, 4],
    [0, 0, 0, 7, 0, 0, 0, 0, 5],
    [0, 0, 0, 1, 0, 0, 0, 3, 6],
    [0, 1, 0, 0, 8, 0, 0, 4, 0],
    [9, 8, 0, 0, 0, 3, 0, 0, 0],
    [3, 0, 0, 0, 0, 1, 0, 0, 0],
    [7, 0, 5, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 2, 5, 0, 0, 0, 8],
]

# Binary variables: x[i,j,k] == 1 if digit k+1 is in cell (i,j)
x = m_sudoku.addVars(9, 9, 9, vtype=GRB.BINARY, name="x")

# Constraint 1: Each cell (i,j) has exactly one number
for i in range(9):
    for j in range(9):
        m_sudoku.addConstr(gp.quicksum(x[i, j, k] for k in range(9)) == 1)

# Constraint 2: Each number appears once in each row
for i in range(9):
    for k in range(9):
        m_sudoku.addConstr(gp.quicksum(x[i, j, k] for j in range(9)) == 1)

# Constraint 3: Each number appears once in each column
for j in range(9):
    for k in range(9):
        m_sudoku.addConstr(gp.quicksum(x[i, j, k] for i in range(9)) == 1)

# Constraint 4: Each number appears once in each 3x3 block
for block_i in range(3):
    for block_j in range(3):
        for k in range(9):
            m_sudoku.addConstr(
                gp.quicksum(
                    x[i, j, k]
                    for i in range(block_i * 3, (block_i + 1) * 3)
                    for j in range(block_j * 3, (block_j + 1) * 3)
                ) == 1
            )

# Constraint 5: Fix values from the initial puzzle
for i in range(9):
    for j in range(9):
        val = puzzle[i][j]
        if val > 0:
            m_sudoku.addConstr(x[i, j, val - 1] == 1)

# Solve the model
m_sudoku.optimize()

# Pretty-print the solution
if m_sudoku.status == GRB.OPTIMAL:
    print("\nSolved Sudoku:")
    for i in range(9):
        row = ""
        for j in range(9):
            for k in range(9):
                if x[i, j, k].X > 0.5:
                    row += f"{k+1} "
                    break
            if j in {2, 5}:
                row += "| "
        print(row)
        if i in {2, 5}:
            print("-" * 21)
else:
    print("No solution found.")

### 4.1 Uniqueness

In general, for a combinatorial problem, to ensure uniqueness of a solution we do the following


1.   Solve the program. If it is not infeasible, retrieve the solution $z^0 \in \{0, 1\}^n$. In particular, there exist $I, J$ indexes such that $I \cap J = \emptyset$, $z^0_i = 0 \; \forall i \in I$ and $z^0_j = 1 \; \forall j \in J$.
2.   Since we are now looking for a solution $z^1 \neq z^0$, we must ensure that either $z^1_i = 1$ for some $i \in I$ or $z^1_j = 0$ for some $j \in J$. This is achieved by adding the constraint
\begin{equation}
  \sum_{i \in I} z_i + \sum_{j \in J} (1- z_j) \geq 1.
\end{equation}
3. We solve the program with this new constraint. If the program is infeasible, $z^0$ was the only feasible solution. If we find a solution $z^1$ with a worst (bigger if we are minimizing, smaller if we are maximizing) objective value, $z^0$ was the only optimal solution. If we find an equivalent solution $z^1$, we can repeat point 2. and add a second constraint in order to find $z^2 \neq z^1$, $z^2 \neq z^0$.


In [ ]:
solution_count = 0

while True:
    m_sudoku.setParam('OutputFlag', 0)
    m_sudoku.optimize()

    if m_sudoku.Status != GRB.OPTIMAL:
        break  # No more solutions

    # Increment count
    solution_count += 1
    print(f"Solution #{solution_count} found.")

    # Collect current solution as a list of (i, j, k) where x[i,j,k] == 1
    current_sol = [(i, j, k) for i in range(9) for j in range(9) for k in range(9)
                   if x[i, j, k].X > 0.5]

    # Complementary set
    comp_sol = [(i, j, k) for i in range(9) for j in range(9) for k in range(9)
                if x[i, j, k].X <= 0.5]

    # Add a no-good cut: prevent same combination from reappearing
    m_sudoku.addConstr(
        gp.quicksum(1 - x[i, j, k] for (i, j, k) in current_sol) +
        gp.quicksum(x[i, j, k] for (i, j, k) in comp_sol) >= 1
    )
print(f"Total solutions found: {solution_count}")

## 5. Combinatorial Auctions
In many auctions, the value that a bidder has for a set of items may not be the sum of the values that he has for individual items. It may be more or it may be less. Examples are equity trading, electricity markets, pollution right auctions and auctions for airport landing slots. To take this into account, combinatorial auctions allow the bidders to submit bids on combinations of items.

Specifically, let $M = \{1, 2, \dots, m\}$ be the set of items that the auctioneer has to sell. A bid is a pair $B_j = (S_j , p_j)$ where $S_j \subseteq M$ is a nonempty set of items and $p_j$ is the price offer for this set. Suppose that the auctioneer has received $n$ bids $B_1, B_2, \dots, B_n$. How should the auctioneer determine the winners in order to maximize his revenue? This can be done by solving an integer program. Let $x_j$ be a $0,1$ variable that takes the value $1$ if bid $B_j$ wins, and $0$ if it looses. The auctioneer maximizes his revenue by solving the integer program:
\begin{align}
\max \quad &\sum_{i=1}^n p_j x_j & \\
\text{s.t.} & \sum_{j : i \in S_j} x_j \leq 1 & i = 1, \dots, m, \\
& x_j \in \{0,1\} & j = 1, \dots, n.
\end{align}
The constraints impose that each item $i$ is sold at most once.

### 5.1. Exercise
Write an ILP program to solve the above problem with the following data.

$B_1 = (\{1\}, 6), \; B_2 = (\{2\}, 3), \; B_3 = (\{3, 4\}, 12), \; B_4 = (\{1,3\}, 12), \; B_5 = (\{2, 4\}, 8), \; B_6 = (\{1, 3, 4\}, 16)$.



In [ ]:
# Bids: list of (set of items, price)
bids = [
    ({1}, 6),
    ({2}, 3),
    ({3, 4}, 12),
    ({1, 3}, 12),
    ({2, 4}, 8),
    ({1, 3, 4}, 16),
]

items = {1, 2, 3, 4}
num_bids = len(bids)

# Create model
model = gp.Model("Combinatorial_Auction")

# Decision variables: x[j] = 1 if bid j is selected
x = model.addVars(num_bids, vtype=GRB.BINARY, name="x")

# Objective: Maximize total price of accepted bids
model.setObjective(gp.quicksum(bids[j][1] * x[j] for j in range(num_bids)), GRB.MAXIMIZE)

# Constraint: Each item can be sold at most once
for i in items:
    model.addConstr(
        gp.quicksum(x[j] for j in range(num_bids) if i in bids[j][0]) <= 1,
        name=f"item_{i}_once"
    )

# Solve
model.optimize()

# Print solution
if model.Status == GRB.OPTIMAL:
    print("\nSelected Bids:")
    total_price = 0
    for j in range(num_bids):
        if x[j].X > 0.5:
            items_in_bid = bids[j][0]
            price = bids[j][1]
            print(f"Bid {j+1}: Items {items_in_bid}, Price {price}")
            total_price += price
    print(f"Total Revenue: {total_price}")
else:
    print("No optimal solution found.")

### 5.2. Multiple objects

In some auctions, there are multiple indistinguishable units of each item for sale. A bid in this setting is defined as $B_j = (\lambda^j_1, \lambda^j_2, \dots, \lambda^j_m; p_j)$ where $\lambda^j_i$ is the desired number of units of item $i$ and $p_j$ is the price offer. The auctioneer maximizes his revenue by solving the integer program:

\begin{align}
\max \quad &\sum_{i=1}^n p_j x_j & \\
\text{s.t.} & \sum_{j : i \in S_j} \lambda^j_i x_j \leq u_i & i = 1, \dots, m, \\
& x_j \in \{0,1\} & j = 1, \dots, n,
\end{align}
where $u_i$ is the number of units of item $i$ for sale.

### 5.3. Exercise
Write an ILP program to solve the multiple object problem by generating some data.

In [ ]:
# Parameters
m = 5  # number of items
n = 10  # number of bids
import numpy as np
np.random.seed(0)

# Supply of each item (random units between 5 and 15)
u = np.random.randint(5, 16, m)

# Generate random bids
# L: n x m matrix where L[j, i] = number of units of item i requested in bid j
L = np.random.randint(0, 4, size=(n, m))  # 0–3 units requested per item
p = np.random.randint(10, 101, size=n)  # price for each bid (10–100)

# Create model
model = gp.Model("MultiUnit_Combinatorial_Auction")

# Binary decision variable: x[j] = 1 if bid j is accepted
x = model.addVars(n, vtype=GRB.BINARY, name="x")

# Objective: maximize total revenue
model.setObjective(gp.quicksum(p[j] * x[j] for j in range(n)), GRB.MAXIMIZE)

# Constraints: supply limits
for i in range(m):
    model.addConstr(
        gp.quicksum(L[j, i] * x[j] for j in range(n)) <= u[i],
        name=f"supply_limit_item_{i}"
    )

# Solve
model.optimize()

# Print results
if model.Status == GRB.OPTIMAL:
    print("\n--- Auction Results ---")
    print(f"Item supply: {u}")
    total = 0
    for j in range(n):
        if x[j].X > 0.5:
            print(f"Bid {j+1}: Price = {p[j]}, Demand = {L[j]}")
            total += p[j]
    print(f"Total Revenue: {total}")
else:
    print("No optimal solution found.")

## 6. Simple portfolio optimization

You currently own a portfolio of eight stocks. Using the Markowitz model, you computed the optimal mean/variance portfolio. The weights of these two portfolios are shown in the following table:

$$
\begin{array}{|r|rrrrrrrr|}
\hline
\textbf{Stock} &  A &  B & C & D & E & F & G & H \\
\hline
\textbf{Your Portfolio} & 0.12 & 0.15 & 0.13 & 0.10 & 0.20 & 0.10 & 0.12 & 0.08\\
\textbf{M/V Portfolio}& 0.02 & 0.05 & 0.25 & 0.06 & 0.18 & 0.10 & 0.22 & 0.12\\
\hline
\end{array}
$$

You would like to rebalance your portfolio in order to be closer to the M/V portfolio. To avoid excessively high transaction costs, you decide to rebalance only three stocks from your portfolio. Let $x_i$ denote the weight of stock $i$ in your rebalanced portfolio. The objective is to minimize the quantity

\begin{equation}
|x_1 - 0.02| + |x_2 - 0.05| + \cdots + |x_8 - 0.12|
\end{equation}

which measures how closely the rebalanced portfolio matches the M/V portfolio.
Formulate this problem as a mixed integer linear program. Note that you will need to introduce new continuous variables in order to linearize the
absolute values and new binary variables in order to impose the constraint
that only three stocks are traded.


In [ ]:
stocks = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
w = [0.12, 0.15, 0.13, 0.10, 0.20, 0.10, 0.12, 0.08]  # current portfolio
mu = [0.02, 0.05, 0.25, 0.06, 0.18, 0.10, 0.22, 0.12]  # mean/variance optimal
n = len(w)

# Model
model = gp.Model("Rebalance_Portfolio")

# Variables
x = model.addVars(n, lb=0, ub=1, name="x")  # new weights
d = model.addVars(n, lb=0, name="d")        # absolute deviation
z = model.addVars(n, vtype=GRB.BINARY, name="z")  # whether stock is traded

M = 1.0  # Large constant for conditional constraint

# Objective: minimize total deviation from M/V portfolio
model.setObjective(gp.quicksum(d[i] for i in range(n)), GRB.MINIMIZE)

# Absolute value linearization
for i in range(n):
    model.addConstr(d[i] >= x[i] - mu[i])
    model.addConstr(d[i] >= mu[i] - x[i])

# Limit to 3 trades
model.addConstr(gp.quicksum(z[i] for i in range(n)) <= 3)

# Conditional trading constraint: only allow deviation from current weight if z_i = 1
for i in range(n):
    model.addConstr(x[i] - w[i] <= M * z[i])
    model.addConstr(w[i] - x[i] <= M * z[i])

# Portfolio weights sum to 1
model.addConstr(gp.quicksum(x[i] for i in range(n)) == 1)

# Solve
model.optimize()

# Print results
if model.Status == GRB.OPTIMAL:
    print("\nRebalanced Portfolio (only 3 stocks changed):")
    for i in range(n):
        xi = x[i].X
        if abs(xi - w[i]) > 1e-6:
            print(f"Stock {stocks[i]}: {w[i]:.2f} -> {xi:.2f} (M/V: {mu[i]:.2f})")
    print(f"\nTotal Deviation from M/V Portfolio: {model.ObjVal:.4f}")
else:
    print("No optimal solution found.")